# Damped Harmonic Oscillator 

Damped harmonic oscillator takes form:

$y'' + 2\gamma y' + \omega_0^2 y = 0$ and has general solution:

$y(t) = A \exp{\left(\left( -\gamma + \sqrt{\gamma^2 -\omega_0^2}\right)t\right)} + B \exp{\left(\left( -\gamma - \sqrt{\gamma^2 -\omega_0^2}\right)t\right)}$

Require $\omega_0^2 > \gamma^2$ for underdamped oscilation. For this scenario, the general solution becomes

$y(t) = C e^{- \gamma t} \left(\exp{i (\sqrt{\omega_0^2 - \gamma^2}) t} + \exp{i (\sqrt{\omega_0^2 - \gamma^2}) t}\right)\\$
$\Rightarrow y(t) = 2C e^{-\gamma t} \cos{\left(\omega t + \phi\right)} \\$

where $\omega = \sqrt{\omega^2_0 -\gamma^2}$ and $\phi$ is some phase offset determined by the initial conditions. We will take $\phi = 0$ so the oscillator is at maximum displacement at $t=0$. 

Aim to make a neural network to approximate the harmonic oscillator function, initialised with random weights and biases.

In [2]:
import torch as t
import numpy as np 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F #contains various functions including activation functions 
import matplotlib.pyplot as plt 

In [1]:
#Analytical Solution
#A = 2C here

def analytical(time, gamma, omega_0, A):
    omega = np.sqrt(omega_0**2 - gamma**2)
    return A*np.exp(-gamma*time)*np.cos(omega*time)